In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [73]:
!ls "/content/gdrive/My Drive/Research/ML for fluid dynamics/RS_AK_code"

__pycache__  solvers.py  TrainingData_SD.ipynb


In [5]:
path = F"/content/gdrive/My Drive/Research/ML for fluid dynamics/RS_AK_code"

In [6]:
import sys
# Append the directory to your python path using sys
sys.path.append(path)

In [53]:
import numpy as np
import solvers
from plotly import graph_objs as go
from plotly.subplots import make_subplots
from tqdm.notebook import trange

In [51]:
#Create a list of parameters to iterate through

#Global parameters:
L = 43
h = 1 # grid spacing
N_g = int(L/h)
N = 1 # Number of Particles
DatasetSize = 100
#Local Parameters
LocalParameterArray = np.zeros((4, DatasetSize))
#Generating Sizes
a_values = np.around(np.arange(0.1, 21.6,0.1), decimals = 1)
LocalParameterArray[0] = np.random.choice(a_values, size = DatasetSize)
# print(LocalParameterArray)

#Generating Positions
#r = [rx, ry, N_g/2]
LocalParameterArray[2:] = np.random.randint(0, N_g, size = (2, DatasetSize))
# print(LocalParameterArray)

#Generating Force amplitude
f_values = np.around(np.arange(0.1, 50.1,0.1), decimals = 1)
LocalParameterArray[1] = np.random.choice(f_values, size = DatasetSize)
# print(LocalParameterArray)


parameters = {'h':h, 'Lx':L, 'Ly':L, 'Lz':L, 'N':N, 'N_g':N_g, 'indexingConvention':'ij' }
sim = solvers.Stokes3D(parameters)

In [68]:
# vImages = np.zeros((DatasetSize*3, 15,15,15))
vImages = np.zeros((DatasetSize, int(1+(L-1)/3),int(1+(L-1)/3)))
v_train = np.zeros((DatasetSize, int(1+(L-1)/3)*int(1+(L-1)/3)))

for i in range(DatasetSize):

    a = LocalParameterArray[0][i]
    par_pl = int(N_g/2)
    r = np.array((LocalParameterArray[2][i], LocalParameterArray[3][i], sim.xx[par_pl]))
    F = np.array((0,0, LocalParameterArray[1][i]))
    sigma = a/(np.sqrt(3))
    vx, vy, vz = sim.vCalc(a, sigma, r, F)


    vx = np.array((sim.vx[:, par_pl, :].reshape(N_g, N_g)))
    vy = np.array((sim.vx[:, par_pl, :].reshape(N_g, N_g)))
    vz = np.array((sim.vz[:, par_pl, :].reshape(N_g, N_g)))
    if i == 0:#Only calculate this the first time
        xx = np.array((sim.x[:, par_pl, :].reshape(N_g, N_g)))
        xx = np.array((sim.y[:, par_pl, :].reshape(N_g, N_g)))
        zz = np.array((sim.z[:, par_pl, :].reshape(N_g, N_g)))

    v_amp = np.sqrt(vx**2+vy**2+vz**2)
    v_amp = v_amp[::3,::3]

    vImages[i,:] = v_amp
    v_train[i,:] = v_amp.flatten()

    # vImages[3*i] = vx[::3,::3]
    # vImages[3*i +1] = vy[::3,::3]
    # vImages[3*i+2] = vz[::3,::3]

  0%|          | 0/100 [00:00<?, ?it/s]

In [69]:
v_train.shape

(100, 225)

In [55]:
v_amp = np.sqrt(vx**2+vy**2+vz**2)
# v_amp = v_amp[::3,::3]
# v_amp = v_amp.flatten()

In [72]:
# Plot
fig = make_subplots(rows=1, cols=2, subplot_titles=('Velocity-Amplitude', 'Force-Amplitude'))

fig.add_heatmap(x=np.arange(0,15),y=np.arange(0,15),z=np.squeeze(vImages[7,:]), row=1, col=1,
                # zmin=0, zmax=1.1,
                showscale=True,
                # zsmooth='best',
                colorscale='Viridis',
                )


fig.layout.xaxis2.title = 'X'
fig.layout.yaxis2.title = 'Y'
# fig.layout.xaxis3.title = 'X'
# fig.layout.yaxis3.title = 'Y'

fig.update_layout(height=525,width=1000)
fig